### **텍스트 요약**

**내장 체인(built-in chains)** 과 [LangGraph](https://langchain-ai.github.io/langgraph/)를 사용하여 **텍스트 요약**을 시연합니다.  

PDF, Notion 페이지, 고객 질문 등 **여러 문서**를 가지고 있고 그 내용을 **요약** 하고 싶다고 가정해 봅시다.

**대규모 언어 모델(LLM)** 은 텍스트를 이해하고 요약하는 데 뛰어난 능력을 가지고 있어 이 작업에 매우 적합합니다.

**검색 증강 생성(Retrieval-Augmented Generation, RAG)** 측면에서 볼 때, 텍스트를 요약하는 것은 많은 수의 검색된 문서에서 **핵심 정보를 추출**하여 LLM에 컨텍스트를 제공하는 데 도움을 줄 수 있습니다.

 **여러 문서의 내용을 LLM을 사용하여 요약하는 방법**을 단계별로 살펴보겠습니다.

![Image description](https://github.com/langchain-ai/langchain/blob/master/docs/static/img/summarization_use_case_1.png?raw=1)

여기에서 다룰 개념은 다음과 같습니다:

- **언어 모델**의 사용.

- **문서 로더**의 사용, 특히 [**WebBaseLoader**](https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.web_base.WebBaseLoader.html)를 사용해 HTML 웹페이지의 콘텐츠를 로드하는 방법.

- **문서를 요약하거나 결합하는 두 가지 방법:**
   1. **Stuff**: 문서를 단순히 프롬프트에 연결합니다.  
   2. **Map-Reduce**: 더 많은 문서 집합에 사용됩니다. 문서를 배치로 나눈 후 각 배치를 요약하고, 마지막으로 요약된 결과를 다시 요약합니다.  


**summarizer**를 구축할 때 중심이 되는 질문은 **어떻게 문서를 LLM의 컨텍스트 윈도우에 전달할 것인가?** 입니다. 이 문제를 해결하기 위한 두 가지 일반적인 접근 방식이 있습니다:

1. **`Stuff`:**  
   - 모든 문서를 **하나의 프롬프트에 "채워 넣는"** 가장 간단한 방법입니다.  
   - 이 방법은 단순하지만 대규모 문서에는 적합하지 않을 수 있습니다.  
<p></p>

2. **`Map-reduce`:**  
   - 먼저 각 문서를 개별적으로 요약하는 **"Map" 단계**를 거칩니다.  
   - 이후, 요약된 결과를 결합하여 최종 요약을 만드는 **"Reduce" 단계**를  수행합니다.  
<p></p>

**`Map-reduce`** 는 특히 **부분 문서(sub-document)** 의 이해가 **이전 맥락에 의존하지 않는 경우** 효과적입니다. 예를 들어, **여러 개의 짧은 문서 모음** 을 요약할 때 유용합니다.  

반면, 소설이나 연속된 흐름이 중요한 긴 텍스트를 요약할 때는 **반복적 개선(iterative refinement)** 접근 방식이 더 효과적일 수 있습니다.

![Image description](https://github.com/langchain-ai/langchain/blob/master/docs/static/img/summarization_use_case_2.png?raw=1)

In [ ]:
# !pip install -qU \
# python-dotenv \
# langchain \
# langchain-community \
# openai \
# anthropic \
# langchain-openai \
# langchain-anthropic \
# langchain-google-genai
# !pip install --upgrade --quiet tiktoken langchain langgraph beautifulsoup4 langchain-community

In [ ]:
# env 파일에서 API 키를 로드

먼저 문서를 불러옵니다. 블로그 게시글을 불러오기 위해 [**WebBaseLoader**](https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.web_base.WebBaseLoader.html)를 사용할 것입니다.

In [ ]:
#  WebBaseLoader 객체 생성
# load(): 지정된 URL의 내용을 가져와 LangChain 문서 형식으로 변환

###  **Stuff: 단일 LLM 호출로 요약하기**

[**`create_stuff_documents_chain`**](https://python.langchain.com/api_reference/langchain/chains/langchain.chains.combine_documents.stuff.create_stuff_documents_chain.html)을 사용할 수 있습니다. 특히 더 큰 **컨텍스트 윈도우(Context Window)**를 지원하는 모델을 사용할 때 유용합니다:

- **128k 토큰 OpenAI `gpt-4o`**  
- **200k 토큰 Anthropic `claude-3-5-sonnet-20240620`**

이 체인은 문서 목록을 가져와 **모든 문서를 하나의 프롬프트로 삽입**한 뒤, 그 프롬프트를 **LLM에 전달**합니다.

In [ ]:
#  프롬프트 정의
# 체인 인스턴스 생성
# 체인 실행 (invoke)

###  **스트리밍**

결과를 **토큰 단위로 스트리밍**할 수도 있습니다.

In [ ]:
# 체인 실행 및 스트리밍

--------------------
### **Map-Reduce: 병렬 처리를 통한 긴 텍스트 요약**

**Map-Reduce 접근법**에서는 먼저 각 문서를 **LLM을 사용하여 개별 요약(map 단계)**한 후, 이러한 요약들을 **하나의 글로벌 요약으로 통합(reduce 단계)**합니다.

**Map 단계**는 일반적으로 입력 문서에 대해 **병렬 처리**됩니다.

**LangGraph**는 `langchain-core`를 기반으로 구축되었으며, [**Map-Reduce**](https://langchain-ai.github.io/langgraph/how-tos/map-reduce/) 워크플로우를 지원하므로 이 문제를 해결하기에 매우 적합합니다:

- **LangGraph**는 각 단계(예: 연속적인 요약)가 **스트리밍될 수 있도록 지원**하여 실행을 더 세밀하게 제어할 수 있습니다.  
- **LangGraph**의 [**체크포인트(checkpointing)**](https://langchain-ai.github.io/langgraph/how-tos/persistence/) 기능은 오류 복구, 인간 검토(human-in-the-loop) 워크플로우 확장, 대화형 애플리케이션 통합을 더욱 쉽게 지원합니다.  
- **LangGraph**는 간단하고 유연하게 수정 및 확장할 수 있습니다. 아래에서 자세히 확인해보겠습니다.

---

### **Map 단계**

먼저 **Map 단계**에 사용할 **프롬프트**를 정의해 보겠습니다.  
이 단계에서는 `stuff` 접근법과 동일한 요약 프롬프트를 사용할 수 있습니다.

In [ ]:
# Map 단계에 사용할 프롬프트 정의

### **Reduce**

문서 매핑 결과를 받아 **하나의 출력으로 통합하는 프롬프트**를 정의합니다.

In [ ]:
# Reduce 프롬프트 템플릿 정의

### **LangGraph를 통한 오케스트레이션**

아래에서는 문서 목록에 대해 요약(Map) 단계를 실행한 다음, 앞서 정의한 프롬프트를 사용해 결과를 통합(Reduce)하는 간단한 애플리케이션을 구현합니다.

**Map-Reduce 흐름**은 LLM의 **컨텍스트 윈도우(context window)**보다 텍스트가 긴 경우 특히 유용합니다. 긴 텍스트의 경우, Reduce 단계에서 요약해야 할 컨텍스트가 **모델의 컨텍스트 윈도우 크기를 초과하지 않도록** 관리하는 메커니즘이 필요합니다.  

여기서는 요약을 **재귀적으로 "압축(collapsing)"**하는 방식을 구현합니다:  
- 입력 텍스트는 **토큰 제한(token limit)**에 따라 분할됩니다.  
- 각 부분(partition)에 대한 요약이 생성됩니다.  
- 이 단계는 요약된 결과의 총 길이가 원하는 한도 내에 도달할 때까지 반복됩니다.  
- 이를 통해 **임의 길이의 텍스트를 효과적으로 요약**할 수 있습니다.

먼저 블로그 게시글을 더 작은 **"하위 문서(sub documents)"**로 분할합니다.

In [ ]:
# 텍스트 분할기 설정
# - chunk_size: 각 하위 문서의 최대 문자 수 (여기서는 1000자)
# - chunk_overlap: 각 청크 간의 중첩 문자 수 (여기서는 0으로 설정)
# 문서를 하위 문서로 분할
# 생성된 하위 문서의 개수 출력

------------
다음으로, **그래프(graph)**를 정의합니다.  

여기서는 **"압축(collapsing)" 단계**를 설명하기 위해 **1,000 토큰**이라는 인위적으로 낮은 최대 토큰 길이를 설정합니다.

In [ ]:
# 최대 토큰 길이 설정
# 문서 길이 계산 함수
def length_function(documents: List[Document]) -> int:
# 전체 그래프 상태 정의
# 문서 내용, 요약, 압축된 요약 및 최종 요약을 포함
class OverallState(TypedDict):
# 각 문서 요약 단계의 상태 정의: 개별 문서를 처리하는 Map 단계에서 사용
class SummaryState(TypedDict):
# 문서 요약 생성 (Map 단계)
# -각 문서를 개별적으로 LLM에 전달하여 요약을 생성합니다.
# 문서에 대한 매핑 논리를 정의. graph의 edge 로 사용 예정.
def map_summaries(state: OverallState):
    # Send 객체의 리스트를 반환
    # 각 Send 객체는 그래프의 노드 이름과 해당 노드로 전달될 상태(state)로 구성
# 요약 결과 수집:  Map 단계에서 생성된 요약을 수집하여 문서 객체로 변환
def collect_summaries(state: OverallState):
# Reduce 단계에서 최종 요약 생성
# 요약 압축 node 추가. 요약을 분할 및 재귀적으로 압축
# 요약 압축 여부 결정 (조건부 Edge)
# 토큰 수가 초과되면 'collapse_summaries'로 이동, 아니면 'generate_final_summary'로 이동
def should_collapse(
# 최종 요약 생성 (Reduce 단계)
# 그래프 구성
# 그래프 엣지(Edge) 정의
# 그래프 컴파일

애플리케이션을 실행할 때 **그래프의 단계 순서**를 관찰하기 위해 스트리밍할 수 있습니다. 아래에서는 간단히 각 단계의 이름을 출력합니다.  

그래프에 **반복(loop)**이 포함되어 있기 때문에, 실행 시 [**recursion_limit**](https://langchain-ai.github.io/langgraph/reference/errors/#langgraph.errors.GraphRecursionError)을 지정하는 것이 유용할 수 있습니다.  

지정된 한도를 초과하면 **GraphRecursionError**가 발생합니다.

In [ ]:
# 그래프를 비동기 스트리밍 방식으로 실행  및 각 단계의 키 출력
# - app.astream: 그래프를 비동기적으로 실행하며 각 단계의 결과를 스트림으로 반환
# - {"contents": [doc.page_content for doc in split_docs]}: 초기 상태로 사용할 문서 내용 목록 전달

------------------
해당 [LangSmith 추적](https://smith.langchain.com/o/351c6cd9-1396-5c74-9478-1ee6a22a6433/projects/p/acec9d4d-4978-4597-adff-789cd42e200f?timeModel=%7B%22duration%22%3A%227d%22%7D&peek=970686e5-ca1c-4f48-ada4-aa182bbb7b53)에서 각각의 노드 아래에 그룹화된 개별 LLM 호출을 확인할 수 있습니다.
